In [ ]:
import os 
import torch
from train_model import test_model, set_seed, get_test_set
from model import FairClassifier

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)

LAMBDA = 0.7

In [ ]:
BATCH_SIZE = 32
NUM_WORKERS = 4

In [ ]:
def evaluate(dataset):
    
    for model_name in os.listdir(dataset):
        model = FairClassifier(dataset).to(device)
        model.load_state_dict(torch.load(os.path.join(dataset, model_name)), strict=False)

        test_set = get_test_set(dataset)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

        test_acc, area_under_curve, area_between_curves_val, \
        margin_plot, precision_plot, ac_plot = test_model(model, test_loader, device)

        plt.show()

        print("Test Accuracy:", test_acc)
        print("Area Under Curve:", area_under_curve)
        print("Area Between Curve:", area_between_curves_val)

## Adult

In [ ]:
evaluate('adult')

## CheXpert

In [ ]:
evaluate('chexpert')

## Celeba

In [ ]:
evaluate('celeba')